In [1]:
import numpy as np
import cv2
import pandas as pd



In [2]:
cap = cv2.VideoCapture('video/video.mp4')
# mendapatkan jumlah frame, fps, lebar, dan tinggi dari video
frames_count, fps, width, height = cap.get(cv2.CAP_PROP_FRAME_COUNT), cap.get(cv2.CAP_PROP_FPS), cap.get(
    cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = int(width)
height = int(height)
print(frames_count, fps, width, height)

# membuat sebuah frame pandas dengan jumlah baris yang sama dengan jumlah frame
df = pd.DataFrame(index=range(int(frames_count)))
df.index.name = "Frame"  # menandai kolom frame

framenumber = 0  # mencatat frame saat ini
carscrossedup = 0  # mencatat mobil yang melintasi jalan ke atas
carscrosseddown = 0  # mencatat mobil yang melintasi jalan ke bawah
carids = []  # daftar kosong untuk menyimpan ID mobil
caridscrossed = []  # daftar kosong untuk menyimpan ID mobil yang sudah melintasi
totalcars = 0  # mencatat jumlah total mobil

fgbg = cv2.createBackgroundSubtractorMOG2()  # membuat pengambil gambar latar belakang

# informasi untuk mulai menyimpan video
ret, frame = cap.read()  # mengimpor gambar
ratio = .5  # rasio ukuran pengubahan ukuran
image = cv2.resize(frame, (0, 0), None, ratio, ratio)  # mengubah ukuran gambar
width2, height2, channels = image.shape


6868.0 30.03550936578534 848 478


In [3]:
while True:

    ret, frame = cap.read()  # mengimpor gambar

    if ret:  # jika ada frame lanjutkan dengan kode

        image = cv2.resize(frame, (0, 0), None, ratio, ratio)  # mengubah ukuran gambar

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # mengubah gambar ke hitam putih

        fgmask = fgbg.apply(gray)  # menggunakan pengambil gambar latar belakang

        # menerapkan berbagai batasan pada fgmask untuk menyaring mobil
        # perlu bermain dengan setelan tersebut hingga mobil dapat diidentifikasi dengan mudah
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # kernel untuk dilakukan pada morphology
        closing = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        dilation = cv2.dilate(opening, kernel)
        retvalbin, bins = cv2.threshold(dilation, 220, 255, cv2.THRESH_BINARY)  # menghapus shadow

        # membuat kontur
        contours, hierarchy = cv2.findContours(bins, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]

        # menggunakan konveks hull untuk membuat poligon di sekitar kontur
        hull = [cv2.convexHull(c) for c in contours]

        # menggambar kontur
        cv2.drawContours(image, hull, -1, (0, 255, 0), 3)

        # garis dibuat untuk menghentikan menghitung kontur, perlu dilakukan karena mobil yang jauh akan menjadi satu kontur besar
        lineypos = 225
        cv2.line(image, (0, lineypos), (width, lineypos), (255, 0, 0), 5)

        # garis y pos dibuat untuk menghitung kontur
        lineypos2 = 250
        cv2.line(image, (0, lineypos2), (width, lineypos2), (0, 255, 0), 5)

        # minimum area untuk kontur
        minarea = 300

        # maksimum area untuk kontur
        maxarea = 50000

        # vektor untuk x dan y lokasi centroid di frame saat ini
        cxx = np.zeros(len(contours))
        cyy = np.zeros(len(contours))

        for i in range(len(contours)):  # mengulangi seluruh kontur dalam frame saat ini

            if hierarchy[0, i, 3] == -1:  # menggunakan hierarchy untuk hanya menghitung kontur induk (tidak termasuk dalam kontur lain)

                area = cv2.contourArea(contours[i])  # menghitung area kontur

                if minarea < area < maxarea:  # area threshold untuk kontur

                    # menghitung centroid dari kontur
                    cnt = contours[i]
                    M = cv2.moments(cnt)
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])

                    if cy > lineypos:  # menghapus kontur yang di atas garis

                        # mengambil titik teratas, kiri, dan lebar dari kontur untuk membuat kotak
                        # x,y adalah kiri atas dan w,h adalah lebar dan tinggi
                        x, y, w, h = cv2.boundingRect(cnt)

                        # membuat kotak di sekitar kontur
                        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

                        # Menuliskan teks centroid untuk memastikan kembali nanti
                        cv2.putText(image, str(cx) + "," + str(cy), (cx + 10, cy + 10), cv2.FONT_HERSHEY_SIMPLEX,
                                    .3, (0, 0, 255), 1)

                        cv2.drawMarker(image, (cx, cy), (0, 0, 255), cv2.MARKER_STAR, markerSize=5, thickness=1,
                                       line_type=cv2.LINE_AA)

                        # menambahkan centroid yang lulus pada kriteria ke dalam list centroid
                        cxx[i] = cx
                        cyy[i] = cy

        # menghapus entri 0 dari list centroid
        cxx = cxx[cxx != 0]
        cyy = cyy[cyy != 0]

        # list kosong untuk nanti menyimpan indices centroid yang di tambahkan ke dataframe
        minx_index2 = []
        miny_index2 = []

        # batas maksimum untuk radius dari centroid dari frame saat ini untuk dianggap sama dengan centroid dari frame sebelumnya
        maxrad = 25

        # Bagian ini mengelola centroid dan menetapkan mereka untuk carid lama atau carid baru

        if len(cxx):  # jika ada centroid dalam area yang ditentukan

            if not carids:  # jika carids kosong

                for i in range(len(cxx)):  # melalui semua centroid

                    carids.append(i)  # menambahkan car id ke list carids kosong
                    df[str(carids[i])] = ""  # menambahkan kolom ke dataframe sesuai carid

                    # menetapkan nilai centroid ke frame (baris) dan carid (kolom) yang sesuai
                    df.at[int(framenumber), str(carids[i])] = [cxx[i], cyy[i]]

                    totalcars = carids[i] + 1  # menambahkan count car

            else:  # jika carids sudah ada

                dx = np.zeros((len(cxx), len(carids)))  # array baru untuk menghitung deltas
                dy = np.zeros((len(cyy), len(carids)))  # array baru untuk menghitung deltas

                for i in range(len(cxx)):  # melalui semua centroid

                    for j in range(len(carids)):  # melalui semua car id yang sudah ada

                        # mengambil centroid dari frame sebelumnya untuk carid tertentu
                        oldcxcy = df.iloc[int(framenumber - 1)][str(carids[j])]

                        # mengambil centroid dari frame saat ini yang tidak selalu sesuai dengan centroid frame sebelumnya
                        curcxcy = np.array([cxx[i], cyy[i]])

                        if not oldcxcy:  # periksa apakah centroid sebelumnya kosong jika arah sudah tidak ada di layar

                            continue  # lanjutkan ke carid berikutnya

                        else:  # hitung delta centroid untuk membandingkan dengan centroid frame saat ini

                            dx[i, j] = oldcxcy[0] - curcxcy[0]
                            dy[i, j] = oldcxcy[1] - curcxcy[1]

                for j in range(len(carids)):  # melalui semua car id saat ini

                    sumsum = np.abs(dx[:, j]) + np.abs(dy[:, j])  # menghitung delta wrt car id

                    # mengambil indeks centroid yang memiliki nilai delta minimum dan ini indeks benar
                    correctindextrue = np.argmin(np.abs(sumsum))
                    minx_index = correctindextrue
                    miny_index = correctindextrue

                    # mengambil delta nilai minimum untuk dibandingkan dengan radius
                    mindx = dx[minx_index, j]
                    mindy = dy[miny_index, j]

                    if mindx == 0 and mindy == 0 and np.all(dx[:, j] == 0) and np.all(dy[:, j] == 0):
                        # periksa apakah minimum nilai adalah 0 dan semua delta adalah nol
                        # delta dapat berupa nol jika centroid tidak bergerak

                        continue  # lanjutkan ke carid berikutnya

                    else:

                        # jika delta nilai adalah kurang dari maksimal radius maka tambahkan centroid ke carid sebelumnya
                        if np.abs(mindx) < maxrad and np.abs(mindy) < maxrad:

                            # tambahkan centroid ke carid yang sudah ada
                            df.at[int(framenumber), str(carids[j])] = [cxx[minx_index], cyy[miny_index]]
                            minx_index2.append(minx_index)  # tambahkan semua indeks yang ditambahkan ke carid ke list
                            miny_index2.append(miny_index)

        currentcars = 0  # current cars on screen
        currentcarsindex = []  # current cars on screen carid index

        for i in range(len(carids)):  # loops through all carids

            if df.at[int(framenumber), str(carids[i])] != '':
                # checks the current frame to see which car ids are active
                # by checking in centroid exists on current frame for certain car id

                currentcars = currentcars + 1  # adds another to current cars on screen
                currentcarsindex.append(i)  # adds car ids to current cars on screen

        for i in range(currentcars):  # loops through all current car ids on screen

            # grabs centroid of certain carid for current frame
            curcent = df.iloc[int(framenumber)][str(carids[currentcarsindex[i]])]

            # grabs centroid of certain carid for previous frame
            oldcent = df.iloc[int(framenumber - 1)][str(carids[currentcarsindex[i]])]

            if curcent:  # if there is a current centroid

                # On-screen text for current centroid
                cv2.putText(image, "Centroid" + str(curcent[0]) + "," + str(curcent[1]),
                            (int(curcent[0]), int(curcent[1])), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 255, 255), 2)

                cv2.putText(image, "ID:" + str(carids[currentcarsindex[i]]), (int(curcent[0]), int(curcent[1] - 15)),
                            cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 255, 255), 2)

                cv2.drawMarker(image, (int(curcent[0]), int(curcent[1])), (0, 0, 255), cv2.MARKER_STAR, markerSize=5,
                               thickness=1, line_type=cv2.LINE_AA)

                if oldcent:  # checks if old centroid exists
                    # adds radius box from previous centroid to current centroid for visualization
                    xstart = oldcent[0] - maxrad
                    ystart = oldcent[1] - maxrad
                    xwidth = oldcent[0] + maxrad
                    yheight = oldcent[1] + maxrad
                    cv2.rectangle(image, (int(xstart), int(ystart)), (int(xwidth), int(yheight)), (0, 125, 0), 1)

                    # checks if old centroid is on or below line and curcent is on or above line
                    # to count cars and that car hasn't been counted yet
                    if oldcent[1] >= lineypos2 and curcent[1] <= lineypos2 and carids[
                        currentcarsindex[i]] not in caridscrossed:

                        carscrossedup = carscrossedup + 1
                        cv2.line(image, (0, lineypos2), (width, lineypos2), (0, 0, 255), 5)
                        caridscrossed.append(
                            currentcarsindex[i])  # adds car id to list of count cars to prevent double counting

                    # checks if old centroid is on or above line and curcent is on or below line
                    # to count cars and that car hasn't been counted yet
                    elif oldcent[1] <= lineypos2 and curcent[1] >= lineypos2 and carids[
                        currentcarsindex[i]] not in caridscrossed:

                        carscrosseddown = carscrosseddown + 1
                        cv2.line(image, (0, lineypos2), (width, lineypos2), (0, 0, 125), 5)
                        caridscrossed.append(currentcarsindex[i])

        # Top left hand corner on-screen text
        cv2.rectangle(image, (0, 0), (250, 100), (255, 0, 0), -1)  # background rectangle for on-screen text

        cv2.putText(image, "Cars in Area: " + str(currentcars), (0, 15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 170, 0), 1)

        cv2.putText(image, "Cars Crossed Up: " + str(carscrossedup), (0, 30), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 170, 0),
                    1)

        cv2.putText(image, "Cars Crossed Down: " + str(carscrosseddown), (0, 45), cv2.FONT_HERSHEY_SIMPLEX, .5,
                    (0, 170, 0), 1)

        cv2.putText(image, "Total Cars Detected: " + str(len(carids)), (0, 60), cv2.FONT_HERSHEY_SIMPLEX, .5,
                    (0, 170, 0), 1)

        cv2.putText(image, "Frame: " + str(framenumber) + ' of ' + str(frames_count), (0, 75), cv2.FONT_HERSHEY_SIMPLEX,
                    .5, (0, 170, 0), 1)

        cv2.putText(image, 'Time: ' + str(round(framenumber / fps, 2)) + ' sec of ' + str(round(frames_count / fps, 2))
                    + ' sec', (0, 90), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 170, 0), 1)

        # displays images and transformations
        cv2.imshow("countours", image)
        cv2.moveWindow("countours", 0, 0)

        cv2.imshow("fgmask", fgmask)
        cv2.moveWindow("fgmask", int(width * ratio), 0)

        cv2.imshow("closing", closing)
        cv2.moveWindow("closing", width, 0)

        cv2.imshow("opening", opening)
        cv2.moveWindow("opening", 0, int(height * ratio))

        cv2.imshow("dilation", dilation)
        cv2.moveWindow("dilation", int(width * ratio), int(height * ratio))

        cv2.imshow("binary", bins)
        cv2.moveWindow("binary", width, int(height * ratio))

        # video.write(image)  # save the current image to video file from earlier

        # adds to framecount
        framenumber = framenumber + 1

        k = cv2.waitKey(int(1000/fps)) & 0xff  # int(1000/fps) is normal speed since waitkey is in ms
        if k == 27:
            break

    else:  # if video is finished then break loop

        break


KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()
